In [ ]:
# --- OPTIMIZED KAGGLE NOTEBOOK SETUP SCRIPT ---
# This script aims to:
# 1. Install system-level OCR tools (Tesseract).
# 2. Handle GPU-specific installations (llama-cpp-python with CUBLAS).
# 3. Resolve common dependency conflicts, especially NumPy compatibility.
# 4. Install core AI/ML libraries.
# 5. Provide clear restart instructions.

import os
import torch

In [ ]:
# Step 1: Suppress Tokenizers Parallelism Warning (Good Practice)
# This prevents a common warning from Hugging Face tokenizers when forking processes.
# ==============================================================================
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("TOKENIZERS_PARALLELISM environment variable set to 'false' to suppress warnings.")


In [ ]:
# Step 2: Install System-Level Dependencies (Tesseract OCR)
# `apt update` first, then `apt install`. Combine into one command for efficiency.
# ==============================================================================
print("\n--- Installing Tesseract OCR and development libraries ---")
!sudo apt update -qq && sudo apt install -qq tesseract-ocr libtesseract-dev

In [ ]:
# Step 3: Initial Check for CUDA and Install GPU-enabled llama-cpp-python
# This section ensures llama-cpp-python is compiled with CUDA support if a GPU is available.
# We uninstall it first to ensure a clean reinstallation with specific CMAKE_ARGS.
# ==============================================================================
print("\n--- Checking CUDA availability and installing llama-cpp-python ---")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print("Uninstalling existing llama-cpp-python for clean CUBLAS build...")
    !pip uninstall llama-cpp-python -y

    print("Installing llama-cpp-python with CUBLAS support...")
    # CMAKE_ARGS="-DLLAMA_CUBLAS=on" is crucial for GPU acceleration with llama-cpp-python
    # --force-reinstall --upgrade ensure a fresh, up-to-date installation
    !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --force-reinstall --upgrade -qqq
    print("llama-cpp-python with CUBLAS installed.")
else:
    print("CUDA not available. Installing CPU-only llama-cpp-python (if not already present)...")
    !pip install llama-cpp-python -qqq # Install CPU version quietly if no GPU


In [ ]:
# Step 4: Resolve NumPy Compatibility & Reinstall Core Data Science Libraries
# This addresses the common 'DataFrame' object has no attribute 'append' (NumPy/Pandas)
# and other potential version mismatches that arise in Kaggle environments.
# ==============================================================================
print("\n--- Resolving NumPy compatibility and reinstalling core libraries ---")

print("Uninstalling potentially conflicting versions of NumPy and related libraries...")
# Uninstall common libraries that might have conflicting NumPy dependencies
# Use -y to auto-confirm uninstallation
!pip uninstall numpy matplotlib pandas scipy scikit-learn -y
# Add any other libraries that frequently cause issues if you encounter them (e.g., pillow, seaborn, accelerate, transformers, bitsandbytes)

print("\nClearing pip cache to ensure fresh downloads...")
!pip cache purge

print("\nInstalling a specific, stable NumPy 1.x version (1.26.4 is often reliable)...")
!pip install numpy==1.26.4 -qqq # -qqq for quiet output

print("\nReinstalling/Upgrading core data science and AI/ML libraries...")
# -U ensures upgrade if already present. These will now install compatible with NumPy 1.26.4
# Consolidate all your necessary Python libraries into one command.
# Removed auto-gptq as it's often part of transformers/optimum, or you'd install it separately.
# Removed bitsandbytes, accelerate, peft, trl, sentencepiece from separate uninstall
# as they will be handled by the transformers[torch] or subsequent reinstall.
!pip install -U -qqq transformers[torch] accelerate auto-gptq optimum pytesseract pandas matplotlib seaborn scipy scikit-learn peft trl sentencepiece

print("\nAll core libraries reinstalled/upgraded, compatible with NumPy 1.26.4.")


In [ ]:
# Step 5: Critical Restart Instruction
# This is absolutely necessary for the new package versions to be loaded into
# the Python runtime, especially after installing system packages or
# deep dependency changes like NumPy.
# ==============================================================================
print("\n" * 2) # Add some space for visibility
print("====================== CRITICAL STEP ======================")
print("=== Please RESTART your Kaggle Notebook session NOW. ===")
print("=== Go to 'File' -> 'Restart Session' in the menu. ===")
print("=== After restarting, re-run all your cells from the beginning. ===")
print("===========================================================")

In [3]:
import os
import shutil

shutil.copy("/kaggle/input/invoices/invoices.db", "/kaggle/working/invoices.db")

'/kaggle/working/invoices.db'

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
exchange_Curr = user_secrets.get_secret("exchange_Curr")
GEMINI_KEY = user_secrets.get_secret("GEMINI_KEY")
HUGGINGFACE_API = user_secrets.get_secret("HUGGINGFACE_API")

In [1]:
import os
import cv2
import pytesseract
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date,datetime
import json
import requests
import time
import google.generativeai as genai
import sqlite3
#from llama_cpp import Llama
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import InferenceClient

In [2]:
from IPython.display import display, HTML
from tqdm.notebook import tqdm # For progress bar in Colab

In [5]:
from huggingface_hub import login

# Paste your Hugging Face token here
login(HUGGINGFACE_API)

In [22]:
# Configure API key
genai.configure(api_key=GEMINI_KEY)

# Initialize the model (e.g., Gemini Pro)
model = genai.GenerativeModel('gemini-2.0-flash')

In [7]:
#Various Agents

class BaseAgent:
    def __init__(self, name):
        self.name = name

    def run(self, input_data):
        raise NotImplementedError("Agent must implement run method")

    def __repr__(self):
        return f"{self.name} Agent"

In [8]:
#OCR AGENT

class OCRAgent(BaseAgent):
    def __init__(self):
        super().__init__("OCR")
        print("Inside OCR init")
        
    def cleaning_text(self,text):
        print("cleaning text OCR")
        #text = re.sub(r'\n+', ' ', text)
        #text = re.sub(r'[^a-zA-Z0-9 ₹.,:/-]', ' ', text)
        #text = re.sub(r'\s+', ' ', text).strip()
        return text.lower()

    def run(self, image_path):
        print("ocr run")
        try:
            img = cv2.imread(image_path)
            filename=image_path.split("/")[-1]
            if img is None:
                raise ValueError(f"Image not loaded properly: {image_path}")

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            """blur = cv2.GaussianBlur(gray, (5, 5), 0)
            _, thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY)"""
            clean_text = pytesseract.image_to_string(gray)
            #print(clean_text)
            clean_text=self.cleaning_text(clean_text)
            return {"filename":filename,"text": clean_text}
        except Exception as e:
            print(f"[ERROR] Failed to process {image_path}: {e}")
            return ""

In [9]:
class SemanticEntityAgentWithGemini(BaseAgent):
    def __init__(self):
        super().__init__("SemanticEntity")
        print("Inside Gemini init")

    def run(self, input_data):
        print("Inside gemini run")
        text = input_data.get("text", "")
        text = [line.strip() for line in text.split("\n") if line.strip()]
        #curr_date = date.today()
        
        prompt = f"""
        Extract the following fields from the invoice text and return them as a JSON object:
        - seller_name
        - invoice_no
        - invoice_date
        - buyer_name
        - total

        Example Input:
        Lopez, Miller and Romero (Only  the name)
        60464 Curtis Gateway
        East Keith, IN 57123

        Invoice Date: 05.08.2007
        Invoice No: 802205

        To: 
        Hayes LLC (Only the name)
        Mercedes Martinez
        960 Hurley Springs North
        Alyssa, RI 49322

        Total: $534.11

        Example Output:
        
        Following things in JSON format of key:value
          "seller": "Lopez, Miller and Romero",
          "invoice_no": "802205",
          "invoice_date": "05/08/2007"(Put into this format "DD/MM/YYYY"),
          "buyer": "Hayes LLC",
          "currency" : "USD(3 Letter Currency code)"
          "total": "534.11 (only float values)"
   
        Now process this input:
        {text}
        """

        # Call the OpenAI API (you'll need to set up your API key first)
        response = model.generate_content(prompt)
        time.sleep(1)

        # ✅ Extract actual text from Gemini response
        output_text = response.candidates[0].content.parts[0].text.strip()

        # ✅ Remove markdown code block if present (like ```json ... ```)
        if output_text.startswith("```json"):
            output_text = output_text.strip("```json").strip("```").strip()

        #print("🔍 Gemini Output:\n", output_text)

        try:
            data = json.loads(output_text)
            #data["text"],data["filename"] = input_data.get("text", ""),input_data.get("filename", "")
            data.update(input_data)
            return data

        except json.JSONDecodeError:
            return {"error": "Failed to parse JSON", "raw_output": output_text}

In [10]:
#Validation Agent
        
class ValidationAgent(BaseAgent):
    def __init__(self):
        super().__init__("Validation")
        print("Inside validation init")

    def run(self, input_data):
        print("Inside validation run")
        missing_fields = []

        #validate input data
        #validate date
        parsed_invoice_date = None
        
        date_formats_to_try = [
            "%d/%m/%Y",  # DD/MM/YYYY (e.g., 15/07/2022)
            "%m/%d/%Y",  # MM/DD/YYYY (e.g., 12/25/2013)
            "%Y-%m-%d",  # YYYY-MM-DD (e.g., 2023-01-31)
            "%d-%m-%Y",  # DD-MM-YYYY (e.g., 31-01-2023)
            "%d.%m.%Y",  # DD.MM.YYYY (e.g., 05.08.2007) - from your example
            "%m.%d.%Y",  # MM.DD.YYYY (e.g., 08.05.2007)
            "%Y/%m/%d",  # YYYY/MM/DD (e.g., 2023/01/31)
        ]
        curr_date=date.today()

        if not input_data["invoice_date"] or not isinstance(input_data["invoice_date"], str) or input_data["invoice_date"].strip() == "":
            input_data["error_message"]="""Date not found. Please verify before submission."""

        else:
            date_str = input_data["invoice_date"].strip()
            for fmt in date_formats_to_try:
                try:
                    parsed_invoice_date = datetime.strptime(date_str, fmt).date()
                    # If parsing is successful, break the loop
                    break
                except ValueError:
                    continue # Try next format if current one fails
            
            if parsed_invoice_date is None:
                # If no format matched, set an error message
                input_data["error_message"] = f"Date format incorrect for '{date_str}'. Expected one of {', '.join(date_formats_to_try)}. Please verify before submission."
            elif parsed_invoice_date > curr_date:
                input_data["error_message"] = "Invoice date is in the future. Please verify before submission."
            # If parsing was successful and date is not in future, no error_message is set for date

        #extract amount and convert it to INR

        currency_code,amount=input_data["currency"],input_data["total"].strip()

        try:
            res = requests.get("https://v6.exchangerate-api.com/v6/f3f6bfc0330eb424583fd63b/latest/INR")
            rates = res.json()["conversion_rates"]
            rate = rates.get(currency_code, None)

            if not rate:
                #return {"error": f"Currency {currency_code} not found in exchange rate API."}
                input_data["Total_in_INR"]=amount
        
            else:
                # Step 5: Convert to INR
                converted_inr = round(float(amount) / rate, 2) if currency_code != "INR" else amount
                input_data["Total_in_INR"] = converted_inr

        except Exception as e:
            return {"error": "Exchange rate fetch failed", "details": str(e)}       


        for field in ["buyer", "seller", "invoice_date", "total"]:
            if field not in input_data:
                missing_fields.append(field)

        if missing_fields:
            input_data["error_message"] = "Following fields missing from image "+" ".join(map(str,missing_fields))

        print("===================Final input of Validation Agent============\n" , input_data)

        return {
            "status": "complete" if not missing_fields else "incomplete",
            "missing": missing_fields,
            "entities": input_data
        }

In [11]:
#Visualizer Agent

class VisualizerAgent(BaseAgent):
    def __init__(self):
        super().__init__("Visualizer")
        print("Inside visual init")

    def draw_boxes(self, image, data, column_text, color, col_name):
        print("Inside draw boxes")
        # Skip if the value is missing or empty
        if not column_text or not isinstance(column_text, str) or column_text.strip() == "":
            print(f"⚠️ Skipping box for '{col_name}' — value is empty or missing.")
            return image  # Just return the unchanged image

        target_words = column_text.lower().split()
        matches = []
        current_match = []

        for i, word in enumerate(data["text"]):
            word_lower = word.strip().lower()
            expected_word = target_words[len(current_match)] if current_match else target_words[0]

            if word_lower == expected_word:
                current_match.append(i)
                if len(current_match) == len(target_words):
                    matches.append(current_match.copy())
                    current_match = []
            else:
                current_match = []

        for match in matches:
            x_coords = [data["left"][i] for i in match]
            y_coords = [data["top"][i] for i in match]
            widths = [data["width"][i] for i in match]
            heights = [data["height"][i] for i in match]

            x_min = min(x_coords)-1
            y_min = min(y_coords)-1
            x_max = max([x_coords[i] + widths[i] for i in range(len(match))])+1
            y_max = max([y_coords[i] + heights[i] for i in range(len(match))])+1

            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
            cv2.putText(image, col_name, (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

    def run(self, input_data):
        print("Inside visual run")
        image_path = input_data["image_path"]
        extracted_entities = input_data["entities"]

        image = cv2.imread(image_path)
        data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

        color_map = {
            "seller": (9, 121, 105),
            "invoice_no": (255, 0, 0),
            "invoice_date": (0, 0, 255),
            "buyer": (0, 255, 255),
            "total": (255, 0, 255)
        }

        for col, color in color_map.items():
            if col in extracted_entities:
                self.draw_boxes(image, data, extracted_entities[col], color, col)

        # Display
        df = pd.DataFrame([extracted_entities])
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image_rgb,df

In [12]:
class SQLiteAgent(BaseAgent):
    def __init__(self, db_path):
        super().__init__("SQLite")
        print("Inside sqlite init")
        self.db_path = db_path
        self.connection = sqlite3.connect(self.db_path)
        self.cursor = self.connection.cursor()

    def insert_invoice(self, df):
        try:
            required_columns = [
                "filename", "seller", "buyer", "invoice_no", "invoice_date",
                "currency", "total", "Total_in_INR", "error_message"
            ]

            # Add any missing columns with None (null)
            for col in required_columns:
                if col not in df.columns:
                    df[col] = None

            # Reorder columns to match table
            df_sql = df[required_columns]

            # Insert to SQLite
            df_sql.to_sql("invoices", self.connection, if_exists="append", index=False)
            self.connection.commit()
            print("✅ Inserted DataFrame into SQLite")

        except Exception as e:
            print(f"[SQLiteAgent] ❌ Failed to insert invoice: {e}")

    def run(self, df):
        self.insert_invoice(df)

In [13]:
class SemanticEntityAgentWithMistral(BaseAgent):
    def __init__(self):
        super().__init__("SemanticMistral")
        print("✅ Initialized SemanticMistral Agent")

        self.model_id = "mistralai/Mistral-7B-Instruct-v0.2"

        client = InferenceClient(
            provider="featherless-ai",
            api_key=HUGGINGFACE_API,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id, use_fast=True)

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            device_map="auto",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )

    def run(self, input_data: dict) -> dict:
        print("📥 Inside Mistral `run` method")
        raw_text = input_data.get("text", "")
        cleaned_text = "\n".join(line.strip() for line in raw_text.split("\n") if line.strip())

        prompt = self.build_prompt(cleaned_text)
        print("🚀 Prompting Mistral...")

        try:
            inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
            outputs = self.model.generate(**inputs, max_new_tokens=512)
            full_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

            print("🧠 Raw Mistral Output:\n", full_output)

            parsed_json = self.extract_json(full_output)
            if parsed_json:
                parsed_json.update(input_data)
                return parsed_json
            else:
                return {"error": "Failed to parse JSON", "raw_output": full_output}

        except Exception as e:
            print("❌ LLM Execution Error:", e)
            return {"error": str(e)}

    def build_prompt(self, invoice_text: str) -> str:
        return f"""
        Extract the following fields from the invoice text and return **only** a JSON object (no explanation, no formatting):
        - seller_name
        - invoice_no
        - invoice_date (format: DD/MM/YYYY)
        - buyer_name
        - total (float only)
        - currency (3-letter code)

        Example Input:
        Lopez, Miller and Romero
        Invoice Date: 05.08.2007
        Invoice No: 802205
        To: Hayes LLC
        Total: $534.11

        Expected Output:
        {{
          "seller": "Lopez, Miller and Romero",
          "invoice_no": "802205",
          "invoice_date": "05/08/2007",
          "buyer": "Hayes LLC",
          "currency": "USD",
          "total": "534.11"
        }}

        Now process this input:
        {invoice_text}
        """

    def extract_json(self, text: str) -> dict | None:
        try:
            # Try to find the last JSON object in the text
            json_matches = list(re.finditer(r"{[\s\S]*?}", text))
            if not json_matches:
                return None

            # Assume the last match is the actual output
            json_text = json_matches[-1].group()
            return json.loads(json_text)
        except Exception as e:
            print("⚠️ JSON parsing failed:", e)
            return None

# Batch Processing using Mistral

In [14]:
print("Initializing Agents")

# Initialize agents
ocr_agent = OCRAgent()
entity_agent = SemanticEntityAgentWithGemini()
batch_entity_agent=SemanticEntityAgentWithMistral()
validator_agent = ValidationAgent()
visualizer_agent = VisualizerAgent()
sqlite_agent = SQLiteAgent("/kaggle/working/invoices.db")

Initializing Agents
Inside OCR init
Inside Gemini init
✅ Initialized SemanticMistral Agent


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Inside validation init
Inside visual init
Inside sqlite init


In [17]:
folder_path="/kaggle/input/invoices"

if not os.path.exists(folder_path):
    print("❌ Error: Specified folder path does not exist even after attempting to create dummy data.")
else:
    all_dfs = []
    files_to_process = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', ".jpeg"))]

    files_to_process=files_to_process[:5] #if any 5 files only
    
    if not files_to_process:
        print(f"⚠️ No image files found in '{folder_path}'. Please check the path and file types.")
    else:
        print(f"\n🚀 Running batch pipeline on {len(files_to_process)} files in '{folder_path}'...")
        # Use tqdm for a Colab-friendly progress bar
        for i, file in enumerate(tqdm(files_to_process, desc="Processing Invoices")):
            image_path = os.path.join(folder_path, file)
            try:
                ocr_output = ocr_agent.run(image_path)
                entity_output = batch_entity_agent.run(ocr_output)
                entity_output["filename"] = file
                validation_output = validator_agent.run(entity_output)
                
                # Ensure validation_output["entities"] is a dictionary for DataFrame creation
                if isinstance(validation_output["entities"], dict):
                    df = pd.DataFrame([validation_output["entities"]])
                    print(df)
                else:
                    # Handle cases where entity_output might not be a dict as expected
                    print(f"⚠️ Skipping {file}: Entity extraction did not return a dictionary. Raw output: {validation_output['entities']}")
                    continue

                sqlite_agent.run(df) # Simulate saving to SQLite
                all_dfs.append(df)
            except Exception as e:
                print(f"⚠️ Failed to process {file}: {e}")

        if all_dfs: # Only concatenate if the list is not empty
            final_df = pd.concat(all_dfs, ignore_index=True)
            print("\n--- Final Consolidated DataFrame ---")
            print(final_df) # Print the final DataFrame
            # You would then typically display or save final_df here
            final_df.to_csv("/kaggle/working/Mistral_output.csv",index=False)
        else:
            print("\nNo DataFrames were successfully processed to consolidate.")


🚀 Running batch pipeline on 5 files in '/kaggle/input/invoices'...


Processing Invoices:   0%|          | 0/5 [00:00<?, ?it/s]

ocr run


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


cleaning text OCR
📥 Inside Mistral `run` method
🚀 Prompting Mistral...
🧠 Raw Mistral Output:
 Extract the following fields from the invoice text and return **only** a JSON object (no explanation, no formatting):
        - seller_name
        - invoice_no
        - invoice_date (format: DD/MM/YYYY)
        - buyer_name
        - total (float only)
        - currency (3-letter code)

        Example Input:
        Lopez, Miller and Romero
        Invoice Date: 05.08.2007
        Invoice No: 802205
        To: Hayes LLC
        Total: $534.11

        Expected Output:
        {
          "seller": "Lopez, Miller and Romero",
          "invoice_no": "802205",
          "invoice_date": "05/08/2007",
          "buyer": "Hayes LLC",
          "currency": "USD",
          "total": "534.11"
        }

        Now process this input:
        weiss inc
0074 nunez circle
port juanshire, al 75552
vat number du96330437548
hart and sons
eduardo hoffman
055 fletcher knolls suite 426
johnmouth, ok 4115

...

cleaning text OCR
📥 Inside Mistral `run` method
🚀 Prompting Mistral...
🧠 Raw Mistral Output:
 Extract the following fields from the invoice text and return **only** a JSON object (no explanation, no formatting):
        - seller_name
        - invoice_no
        - invoice_date (format: DD/MM/YYYY)
        - buyer_name
        - total (float only)
        - currency (3-letter code)

        Example Input:
        Lopez, Miller and Romero
        Invoice Date: 05.08.2007
        Invoice No: 802205
        To: Hayes LLC
        Total: $534.11

        Expected Output:
        {
          "seller": "Lopez, Miller and Romero",
          "invoice_no": "802205",
          "invoice_date": "05/08/2007",
          "buyer": "Hayes LLC",
          "currency": "USD",
          "total": "534.11"
        }

        Now process this input:
        mr. d.i.¥. (kuchat) sdn bhd
co-reg:750441-w
lot 1851-a & 1851-b, jalan kpb 6,
kawasan perindustrian balakong,
43300 seri kembangan, selangor
(gst id no :000

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


cleaning text OCR
📥 Inside Mistral `run` method
🚀 Prompting Mistral...
🧠 Raw Mistral Output:
 Extract the following fields from the invoice text and return **only** a JSON object (no explanation, no formatting):
        - seller_name
        - invoice_no
        - invoice_date (format: DD/MM/YYYY)
        - buyer_name
        - total (float only)
        - currency (3-letter code)

        Example Input:
        Lopez, Miller and Romero
        Invoice Date: 05.08.2007
        Invoice No: 802205
        To: Hayes LLC
        Total: $534.11

        Expected Output:
        {
          "seller": "Lopez, Miller and Romero",
          "invoice_no": "802205",
          "invoice_date": "05/08/2007",
          "buyer": "Hayes LLC",
          "currency": "USD",
          "total": "534.11"
        }

        Now process this input:
        invoice no: 51109338
date of issue:
seller:
andrews, kirby and valdez
58861 gonzalez prairie
lake daniellefurt, in 57228
tax id: 945-82-2137
iban: gb75mcrl0

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


cleaning text OCR
📥 Inside Mistral `run` method
🚀 Prompting Mistral...
🧠 Raw Mistral Output:
 Extract the following fields from the invoice text and return **only** a JSON object (no explanation, no formatting):
        - seller_name
        - invoice_no
        - invoice_date (format: DD/MM/YYYY)
        - buyer_name
        - total (float only)
        - currency (3-letter code)

        Example Input:
        Lopez, Miller and Romero
        Invoice Date: 05.08.2007
        Invoice No: 802205
        To: Hayes LLC
        Total: $534.11

        Expected Output:
        {
          "seller": "Lopez, Miller and Romero",
          "invoice_no": "802205",
          "invoice_date": "05/08/2007",
          "buyer": "Hayes LLC",
          "currency": "USD",
          "total": "534.11"
        }

        Now process this input:
        invoice no: 72128555
date of issue:
seller:
obrien group
6217 boyd ville apt. 758
robbinsberg, az 54997
tax id: 905-82-5399
iban: gb71qmaa96406297569205
ite

# Batch Processing or single processing through Gemini

In [23]:
folder_path="/kaggle/input/invoices"

if not os.path.exists(folder_path):
    print("❌ Error: Specified folder path does not exist even after attempting to create dummy data.")
else:
    all_dfs = []
    files_to_process = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', ".jpeg"))]

    files_to_process=files_to_process[:5] #if any 5 files only
    
    if not files_to_process:
        print(f"⚠️ No image files found in '{folder_path}'. Please check the path and file types.")
    else:
        print(f"\n🚀 Running batch pipeline on {len(files_to_process)} files in '{folder_path}'...")
        # Use tqdm for a Colab-friendly progress bar
        for i, file in enumerate(tqdm(files_to_process, desc="Processing Invoices")):
            image_path = os.path.join(folder_path, file)
            try:
                ocr_output = ocr_agent.run(image_path)
                entity_output = entity_agent.run(ocr_output)
                entity_output["filename"] = file
                validation_output = validator_agent.run(entity_output)
                
                # Ensure validation_output["entities"] is a dictionary for DataFrame creation
                if isinstance(validation_output["entities"], dict):
                    df = pd.DataFrame([validation_output["entities"]])
                    print(df)
                else:
                    # Handle cases where entity_output might not be a dict as expected
                    print(f"⚠️ Skipping {file}: Entity extraction did not return a dictionary. Raw output: {validation_output['entities']}")
                    continue

                sqlite_agent.run(df) # Simulate saving to SQLite
                all_dfs.append(df)
            except Exception as e:
                print(f"⚠️ Failed to process {file}: {e}")
            time.sleep(1) #to prevent multiple requests per second
 
        if all_dfs: # Only concatenate if the list is not empty
            final_df = pd.concat(all_dfs, ignore_index=True)
            print("\n--- Final Consolidated DataFrame ---")
            print(final_df) # Print the final DataFrame
            # You would then typically display or save final_df here
            final_df.to_csv("/kaggle/working/Gemini_output.csv",index=False)
        else:
            print("\nNo DataFrames were successfully processed to consolidate.")


🚀 Running batch pipeline on 5 files in '/kaggle/input/invoices'...


Processing Invoices:   0%|          | 0/5 [00:00<?, ?it/s]

ocr run
cleaning text OCR
Inside gemini run
Inside validation run
===================Final input of Validation Agent============
 {'seller_name': 'weiss inc', 'invoice_no': '774800', 'invoice_date': '15/07/2022', 'buyer_name': 'hart and sons', 'currency': 'GBP', 'total': '112.5', 'filename': '007.png', 'text': 'weiss inc\n\n0074 nunez circle\n\nport juanshire, al 75552\n\nvat number du96330437548\n\nhart and sons\n\neduardo hoffman\n\n055 fletcher knolls suite 426\njohnmouth, ok 41153\nargentina\n\nitem _ description\n\n7 morph distributed bandwidth\n\n3 streamline 24/7 models\n\n8 generate bleeding-edge functionalities\n10 evolve intuitive markets\n\n2 innovate integrated applications\n\n5 visualize efficient experiences\n\nunit cost\n97.79\n11.12\n42.63\n20.3\n\n64.9\n\n52.08\n\ninvoice# 774800\ndate 15.07.2022\namount due £186.87\n\nquantity line total\n\n9.39 480.24\n8.52 38.23\n4.86 231.69\n4.59 250.69\n8.79 583.0\n1.04 806.39\n\nsubtotal 129.78\ndiscount -10.45% -29.51\ntotal £11